# Biosignal processing


#### Dependencies

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
import neurokit2 as nk
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import systole
import plotly
import peakdet
import heartpy as hp

In [ ]:
%pylab inline
sns.set_style('darkgrid')
pylab.rcParams['figure.figsize'] = (12.0, 6.0)  
plt.rcParams.update({'font.size': 14})

In [ ]:
%run ~/cneuromod/ds_prep/physio/preproc/neuromod_process.py

In [ ]:
filename = "/scratch/flesp/physio_data/friends/sub-05/ses-013/neuromod_video52021-02-15T16_01_28_08"

In [ ]:
filename[filename.rfind('/')+1:]

In [ ]:
json=pd.read_json(f"{filename}.json")

In [ ]:
fs = json['SamplingFrequency'][0]

In [ ]:
bio_df = pd.read_csv(f"{filename}.tsv.gz",
                     sep='\t',
                     compression='gzip',
                     header=None,
                     names=json.Columns)

In [ ]:
sig = nk.signal_filter(bio_df['PPG'], fs, lowcut=0.5, highcut=8, order=3)
detrend = nk.signal_detrend(bio_df['PPG'], order=2 )

In [ ]:
sliced = np.array(bio_df['PPG'][2500000:3500000])

In [ ]:
info = nk.ppg_findpeaks(sliced, fs, show=True)

In [ ]:
info = nk.ppg_findpeaks(sig[2500000:3500000], fs, show=True)

In [ ]:
info = nk.ppg_findpeaks(detrend[2500000:3500000], fs, show=True)

In [ ]:
wd, m = hp.process(sig, fs)

In [ ]:
hp.plotter(wd, m, show=True)

In [ ]:
m

In [ ]:
wd.keys()

In [ ]:
signals, info_corrected = neuromod_ppg_process(bio_df['PPG'], fs)

In [ ]:
signals.head()

In [ ]:
info_corrected.keys()

In [ ]:
info_corrected['rejected_segments']

In [ ]:
info_corrected['extra']

In [ ]:
info_corrected['ectopic']

In [ ]:
plot = systole.plots.plot_raw(signals['PPG_Clean'], backend='bokeh', show_heart_rate=True, show_artefacts=True, sfreq=fs)

In [ ]:
from bokeh.io import output_notebook
from bokeh.plotting import show, output_file, save
output_notebook()

show(plot)

In [ ]:
plot_rr = systole.plots.plot_rr(info_corrected['clean_rr_systole'],input_type='rr_ms',backend='bokeh', show_artefacts=True)

In [ ]:
output_notebook()

show(plot_rr)

In [ ]:
plot_rr = systole.plots.plot_rr(info_corrected['clean_rr_hp'],input_type='rr_ms',backend='bokeh', show_artefacts=True)

In [ ]:
output_notebook()

show(plot_rr)

In [ ]:
bio_df, bio_info = neuromod_bio_process(df=bio_df, sampling_rate=fs)

In [ ]:
bio_df.keys()

In [ ]:
plt.plot(nk.standardize(bio_df[['RSP_Rate', 'RSP_Clean']][2500000:3500000]))

In [ ]:
rate = nk.ppg_rate(info_corrected['PPG_Peaks'], sampling_rate=fs, desired_length=len(bio_df))

In [ ]:
plt.plot(nk.standardize(rate[2500000:3500000]))